In [1]:
# Uncomment and run to reload libs
# import importlib, utils
# importlib.reload(utils)
# import importlib, utils.miscellaneous
# importlib.reload(utils.miscellaneous)


from itertools import combinations

import pandas as pd

from utils import (
    logging,
    COMBO_MIN_NUM,
    ALL_DATE_COMBO_FILE,
)

from utils.reader import AminoAcidComboReader
from utils.miscellaneous import sort_mut_by_pos


In [2]:
aa_combo_reader: AminoAcidComboReader = AminoAcidComboReader()
all_data = aa_combo_reader.all_data()


In [ ]:
all_date_combo = []
existing_future_combo = set()
ac_group: pd.DataFrame
c_date_group: pd.DataFrame
for c_date, c_date_group in all_data.groupby("Date", sort=True):
    logging.info(f"Future date: {c_date}")
    c_date_combo = set()
    for ac, ac_group in c_date_group.groupby("Accession", sort=False):
        for i in combinations(ac_group["AA"].values, COMBO_MIN_NUM):
            c_date_combo.add(frozenset(i))

    # Reomve combo in the previous dates
    c_date_combo = c_date_combo.difference(existing_future_combo)
    existing_future_combo = existing_future_combo.union(c_date_combo)

    if len(c_date_combo):
        c_date_combo = pd.DataFrame(c_date_combo)
        c_date_combo = c_date_combo.rename(columns={0: "Accession", 1: "Mut"})
        c_date_combo = sort_mut_by_pos(c_date_combo, "Accession", "Mut")
        c_date_combo["Date"] = c_date
        all_date_combo.append(c_date_combo)


In [ ]:
pd.concat(all_date_combo).to_csv(ALL_DATE_COMBO_FILE, index=False)
logging.info(f"{ALL_DATE_COMBO_FILE} saved!")
